In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm

In [2]:
pnl = 0
DATA_DIR       = './data'
SYMBOLS_CSV    = 'symbols.csv'
WINDOW_SIZE    = 183
WIDE_SIZE      = 303
VOLUME_THRESHOLD = 100_000
NOTIONAL       = 100.0

In [3]:
def prepare_data():
    symbols = pd.read_csv(SYMBOLS_CSV)['symbol'].iloc[:-4].tolist()
    data_dict = {}
    for sym in symbols:
        df = pd.read_csv(os.path.join(DATA_DIR, f'{sym}.csv'))
        df['open_time'] = pd.to_datetime(df['open_time'])
        df[['open','high','low','close','volume']] = df[['open','high','low','close','volume']].apply(pd.to_numeric, errors='coerce')
        df.sort_values('open_time', inplace=True)
        df.reset_index(drop=True, inplace=True)
        df['time_index'] = (df['open_time'] - df['open_time'].iloc[0]).dt.total_seconds()
        data_dict[sym] = df
    return data_dict

In [4]:
def create_model(df):
    X = sm.add_constant(df['time_index'])
    y = df['close']
    return sm.OLS(y, X).fit()

In [5]:
def get_predictions(df, model, alpha=0.01):
    X_new = sm.add_constant(df['time_index'])
    return model.get_prediction(X_new).summary_frame(alpha=alpha)

In [6]:
def get_slope(df, pred_summary):
    x0, xN = df['time_index'].iloc[0], df['time_index'].iloc[-1]
    y0, yN = pred_summary['mean'].iloc[0], pred_summary['mean'].iloc[-1]
    pct_change = (yN - y0) / y0 * 100
    minutes = (xN - x0) / 60
    return pct_change / minutes if minutes > 0 else 0

In [7]:
def get_lower_series(lower_series, horizon=WIDE_SIZE):
    n = len(lower_series)
    slope = (lower_series.iloc[-1] - lower_series.iloc[0]) / (n - 1)
    intercept = lower_series.iloc[0]
    x = np.arange(horizon)
    return pd.Series(intercept + slope * x)

In [8]:
def draw(window, lower_preds, ticker, sell_time, sell_price,buy_time,buy_price):
    df = window.copy()
    last_price = df['close'].iloc[-1]
    last_time = df['open_time'].iloc[-1]
    last_lower = lower_preds.iloc[-1]

    plt.figure(figsize=(14, 6))
    plt.plot(df['open_time'], df['close'], label='Real Price',color='black',linewidth=1)
    plt.plot(df['open_time'], lower_preds.iloc[:len(df)], '--', label='Lower CI Projection')
    plt.axvline(x=df['open_time'].iloc[183], color='red', linestyle='--', label='Knowledge Base')
    plt.scatter([sell_time], [sell_price], color='red', label='Signal Price')
    plt.scatter([buy_time], [buy_price], color='green', label='Signal Price')
    
    plt.text(last_time, last_price, f' Last Price: {last_price:.6f}', color='orange', fontsize=10)
    plt.text(last_time, last_lower, f' ↓ {last_lower:.6f}', color='red', fontsize=10)
    
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.title(f'Sell Signal - {ticker}')
    plt.legend()
    plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

stop_slope = []
tp_slope = []


In [9]:
if __name__ == '__main__':
    z =1
    global pnl
    data = prepare_data()
    cooldown_bars = 10

    last_signal = { sym: None for sym in data.keys() }

    for sym, df_full in data.items():
        print(z,sym)
        z+=1
        n = len(df_full)
        for i in range(n - WIDE_SIZE + 1):
            last = last_signal[sym]
            if last is not None and i < last + cooldown_bars:
                continue

            window      = df_full.iloc[i : i+WINDOW_SIZE]
            test_window = df_full.iloc[i : i+WIDE_SIZE]

            if window['close'].nunique() <= 1:
                continue

            model = create_model(window)
            if model.rsquared <= 0.90:
                continue

            preds = get_predictions(window, model, alpha=0.01)
            lower_proj = get_lower_series(preds['obs_ci_lower'])

            if get_slope(window,preds)<0:
                continue
            signal_found = False
            for k in range(WINDOW_SIZE-1, WIDE_SIZE-1):
                prev_thr  = lower_proj.iloc[k]
                curr_p    = test_window['close'].iloc[k]
                next_p    = test_window['close'].iloc[k+1]
                next_thr  = lower_proj.iloc[k+1]
                
                if curr_p > prev_thr and next_p < next_thr:
                    sell_rel     = k+1
                    sell_bar     = i + sell_rel
                    sell_time    = df_full['open_time'].iloc[sell_bar]
                    sell_price   = df_full['close'].iloc[sell_bar]

                    for t in range(k, WIDE_SIZE-1):
                        next_low  = test_window['low'].iloc[t+1]
                        next_high = test_window['high'].iloc[t+1]

                        if next_low < sell_price * 0.995:
                            buy_rel   = t+1
                            buy_bar   = i + buy_rel
                            buy_time  = df_full['open_time'].iloc[buy_bar]
                            buy_price = df_full['close'].iloc[buy_bar]
                            pnl += 0.05
                            print(f'took profit @{buy_price:.6f}   pnl: {round(pnl,2)}')
                            print(get_slope(window,preds))
                            tp_slope.append(get_slope(window,preds))
                            #draw(test_window, lower_proj, sym,sell_time, sell_price,buy_time, buy_price)
                            break

                        elif next_high > sell_price * 1.005:
                            buy_rel   = t+1
                            buy_bar   = i + buy_rel
                            buy_time  = df_full['open_time'].iloc[buy_bar]
                            buy_price = df_full['close'].iloc[buy_bar]
                            pnl -= 0.05
                            print(f'stop loss @{buy_price:.6f}   pnl: {round(pnl,2)}')
                            print(get_slope(window,preds))
                            stop_slope.append(get_slope(window,preds))
                            #draw(test_window, lower_proj, sym,sell_time, sell_price,buy_time, buy_price)
                            break

                    last_signal[sym] = sell_bar
                    signal_found     = True
                    break   

            if signal_found:
                continue

1 BTCUSDT
took profit @104791.600000   pnl: 0.05
0.003310771810876682
2 ETHUSDT
3 BCHUSDT
4 XRPUSDT
5 LTCUSDT
6 TRXUSDT
7 ETCUSDT
took profit @16.731000   pnl: 0.1
0.015480306489716987
8 LINKUSDT
took profit @13.084000   pnl: 0.15
0.009597883143166883
stop loss @12.690000   pnl: 0.1
0.01047590883010728
9 XLMUSDT
10 ADAUSDT
took profit @0.615000   pnl: 0.15
0.010482774120189043
11 XMRUSDT
took profit @314.240000   pnl: 0.2
0.019688708306487435
took profit @314.340000   pnl: 0.25
0.015661314449453012
12 DASHUSDT
took profit @19.840000   pnl: 0.3
0.008550086996180664
stop loss @19.990000   pnl: 0.25
0.011271648844319125
13 ZECUSDT
took profit @41.410000   pnl: 0.3
0.009259536111920385
14 XTZUSDT
15 BNBUSDT
16 ATOMUSDT
17 ONTUSDT
took profit @0.123000   pnl: 0.35
0.008048880785558684
stop loss @0.124000   pnl: 0.3
0.015230037911830228
18 IOTAUSDT
took profit @0.163900   pnl: 0.35
0.009040746796857731
19 BATUSDT
20 VETUSDT
took profit @0.021620   pnl: 0.4
0.009023046836252812
stop loss @0.0

In [10]:
print(stop_slope)

[np.float64(0.01047590883010728), np.float64(0.011271648844319125), np.float64(0.015230037911830228), np.float64(0.008814465269949891), np.float64(0.019759210604560568), np.float64(0.012962482486012894), np.float64(0.01101771731806287), np.float64(0.010615179856734372), np.float64(0.005696416320224395), np.float64(0.019170468563068983), np.float64(0.03237839518234809), np.float64(0.012524749757158504), np.float64(0.01018294299133773), np.float64(0.007507992863306348), np.float64(0.020380368437706825), np.float64(0.029571385012135475), np.float64(0.019119495110441115), np.float64(0.023541477979298352), np.float64(0.017125406464755164), np.float64(0.00956536391266551), np.float64(0.01748311361687425), np.float64(0.018071489089219767), np.float64(0.012034139178917238), np.float64(0.041248999049035354), np.float64(0.008200070387407282), np.float64(0.016511521632956223), np.float64(0.010676064442516734), np.float64(0.015466974386656746), np.float64(0.021152163069365376), np.float64(0.019706

In [ ]:
stop = pd.DataFrame(stop_slope)
tp = pd.DataFrame(tp_slope)
